In [18]:
# 30 min classification coding in python

# from Bio import Entrez  # importer Bio library og modulet entrez
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
import nltk
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn import svm
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from numpy import mean
from numpy import std

from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix


In [3]:
df = pd.read_csv('data.csv')

display(df)

,Unnamed: 0,title,rating,name,num_ratings,num_reviews,num_followers,synopsis,genre
0,0,Sapiens: A Brief History of Humankind,4.39,Yuval Noah Harari,"8,06,229","46,149",30.5k,"100,000 years ago, at least six human species ...",history
1,1,"Guns, Germs, and Steel: The Fates of Human Soc...",4.04,Jared Diamond,"3,67,056","12,879","6,538","""Diamond has written a book of remarkable scop...",history
2,2,A People's History of the United States,4.07,Howard Zinn,"2,24,620","6,509","2,354","In the book, Zinn presented a different side o...",history
3,3,"The Devil in the White City: Murder, Magic, an...",3.99,Erik Larson,"6,13,157","36,644",64.2k,Author Erik Larson imbues the incredible event...,history
4,4,The Diary of a Young Girl,4.18,Anne Frank,"33,13,033","35,591","4,621",Discovered in the attic in which she spent the...,history
...,...,...,...,...,...,...,...,...,...
1534,1534,Hounded,4.09,Kevin Hearne,"83,827","7,203","11,905","Atticus O’Sullivan, last of the Druids, lives ...",fantasy
1535,1535,Charlie and the Chocolate Factory,4.15,Roald Dahl,"7,75,001","14,252","22,897",Charlie Bucket's wonderful adventure begins wh...,fantasy
1536,1536,Red Rising,4.25,Pierce Brown,"3,10,138","30,388","30,510","""I live for the dream that my children will be...",fantasy
1537,1537,Frostbite,4.26,Richelle Mead,"3,37,538","12,435","66,448","Rose loves Dimitri, Dimitri might love Tasha, ...",fantasy


In [4]:
df["synopsis"] = df["synopsis"].str.lower()
display(df["synopsis"])

0       100,000 years ago, at least six human species ...
1       "diamond has written a book of remarkable scop...
2       in the book, zinn presented a different side o...
3       author erik larson imbues the incredible event...
4       discovered in the attic in which she spent the...
                              ...                        
1534    atticus o’sullivan, last of the druids, lives ...
1535    charlie bucket's wonderful adventure begins wh...
1536    "i live for the dream that my children will be...
1537    rose loves dimitri, dimitri might love tasha, ...
1538    the prince of no value\nbrishen khaskem, princ...
Name: synopsis, Length: 1539, dtype: object

In [8]:
df["synopsis"]= df.apply(lambda x: re.sub("[\"'’]","",x["synopsis"]),axis=1) 
display(df["synopsis"])

0       100,000 years ago, at least six human species ...
1       diamond has written a book of remarkable scope...
2       in the book, zinn presented a different side o...
3       author erik larson imbues the incredible event...
4       discovered in the attic in which she spent the...
                              ...                        
1534    atticus osullivan, last of the druids, lives p...
1535    charlie buckets wonderful adventure begins whe...
1536    i live for the dream that my children will be ...
1537    rose loves dimitri, dimitri might love tasha, ...
1538    the prince of no value\nbrishen khaskem, princ...
Name: synopsis, Length: 1539, dtype: object

In [5]:
df["synopsis_proc"]= df.apply(lambda x: word_tokenize(x["synopsis"]),axis=1)
display(df["synopsis_proc"])

0       [100,000, years, ago, ,, at, least, six, human...
1       [``, diamond, has, written, a, book, of, remar...
2       [in, the, book, ,, zinn, presented, a, differe...
3       [author, erik, larson, imbues, the, incredible...
4       [discovered, in, the, attic, in, which, she, s...
                              ...                        
1534    [atticus, o, ’, sullivan, ,, last, of, the, dr...
1535    [charlie, bucket, 's, wonderful, adventure, be...
1536    [``, i, live, for, the, dream, that, my, child...
1537    [rose, loves, dimitri, ,, dimitri, might, love...
1538    [the, prince, of, no, value, brishen, khaskem,...
Name: synopsis_proc, Length: 1539, dtype: object

In [10]:
nltk.download('punkt')
table = str.maketrans('', '', string.punctuation)
df["synopsis_proc"] = df.apply(lambda x: [w.translate(table) for w in x["synopsis_proc"]], axis=1)
display(df["synopsis_proc"])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rasmu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0       [100000, years, ago, , at, least, six, human, ...
1       [diamond, has, written, a, book, of, remarkabl...
2       [in, the, book, , zinn, presented, a, differen...
3       [author, erik, larson, imbues, the, incredible...
4       [discovered, in, the, attic, in, which, she, s...
                              ...                        
1534    [atticus, osullivan, , last, of, the, druids, ...
1535    [charlie, buckets, wonderful, adventure, begin...
1536    [i, live, for, the, dream, that, my, children,...
1537    [rose, loves, dimitri, , dimitri, might, love,...
1538    [the, prince, of, no, value, brishen, khaskem,...
Name: synopsis_proc, Length: 1539, dtype: object

In [6]:
df["synopsis_proc"] = df.apply(lambda x: [w for w in x["synopsis_proc"] if w.isalpha()], axis=1)
display(df["synopsis_proc"])

0       [years, ago, at, least, six, human, species, i...
1       [diamond, has, written, a, book, of, remarkabl...
2       [in, the, book, zinn, presented, a, different,...
3       [author, erik, larson, imbues, the, incredible...
4       [discovered, in, the, attic, in, which, she, s...
                              ...                        
1534    [atticus, o, sullivan, last, of, the, druids, ...
1535    [charlie, bucket, wonderful, adventure, begins...
1536    [i, live, for, the, dream, that, my, children,...
1537    [rose, loves, dimitri, dimitri, might, love, t...
1538    [the, prince, of, no, value, brishen, khaskem,...
Name: synopsis_proc, Length: 1539, dtype: object

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df["synopsis_proc"] = df.apply(lambda x: [w for w in x["synopsis_proc"] if not w in stop_words], axis=1)
display(df["synopsis_proc"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rasmu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       [years, ago, least, six, human, species, inhab...
1       [diamond, written, book, remarkable, scope, on...
2       [book, zinn, presented, different, side, histo...
3       [author, erik, larson, imbues, incredible, eve...
4       [discovered, attic, spent, last, years, life, ...
                              ...                        
1534    [atticus, osullivan, last, druids, lives, peac...
1535    [charlie, buckets, wonderful, adventure, begin...
1536    [live, dream, children, born, free, says, like...
1537    [rose, loves, dimitri, dimitri, might, love, t...
1538    [prince, value, brishen, khaskem, prince, kai,...
Name: synopsis_proc, Length: 1539, dtype: object

In [7]:
df["synopsis_proc_res"] = df.apply(lambda x:' '.join(x["synopsis_proc"]), axis=1)
display(df["synopsis_proc_res"])

0       years ago at least six human species inhabited...
1       diamond has written a book of remarkable scope...
2       in the book zinn presented a different side of...
3       author erik larson imbues the incredible event...
4       discovered in the attic in which she spent the...
                              ...                        
1534    atticus o sullivan last of the druids lives pe...
1535    charlie bucket wonderful adventure begins when...
1536    i live for the dream that my children will be ...
1537    rose loves dimitri dimitri might love tasha an...
1538    the prince of no value brishen khaskem prince ...
Name: synopsis_proc_res, Length: 1539, dtype: object

In [8]:
with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

In [9]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df = pickle.load(f)

In [10]:
df["synopsis_proc_res"]

0       years ago at least six human species inhabited...
1       diamond has written a book of remarkable scope...
2       in the book zinn presented a different side of...
3       author erik larson imbues the incredible event...
4       discovered in the attic in which she spent the...
                              ...                        
1534    atticus o sullivan last of the druids lives pe...
1535    charlie bucket wonderful adventure begins when...
1536    i live for the dream that my children will be ...
1537    rose loves dimitri dimitri might love tasha an...
1538    the prince of no value brishen khaskem prince ...
Name: synopsis_proc_res, Length: 1539, dtype: object

In [11]:

# splitting dataframe by row index
df_train = df.iloc[:1230,:]
df_test = df.iloc[1230:,:]

In [13]:
vec = TfidfVectorizer()
x = vec.fit_transform(df["synopsis_proc_res"])

In [14]:
svd = TruncatedSVD(n_components=100)
res = svd.fit_transform(x)

In [15]:
y = df["genre"].values
X = res
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo')

In [16]:
metrics = cross_validate(model, res, y, scoring=['precision_macro', 'recall_macro'], cv=cv, n_jobs=-1)

print('Precision: %.3f (%.3f)' % (mean(metrics["test_precision_macro"]), std(metrics["test_precision_macro"])))
print('Recall: %.3f (%.3f)' % (mean(metrics["test_recall_macro"]), -std(metrics["test_recall_macro"])))

Precision: 0.581 (0.053)
Recall: 0.454 (-0.039)
